In [40]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.float_format', lambda x: '%.10f' % x)
from random import uniform, randint
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, cross_val_score, RandomizedSearchCV
import xgboost as xgb

# Leitura das bases de treino/validação e teste. Foram consideradas apenas as variáveis que existem na base de teste.
df = pd.read_csv(r'train.csv',index_col=0)
df_test = pd.read_csv(r'test.csv')
df.reset_index(inplace=True)
cols = list(df_test.columns)
cols.append('NU_NOTA_MT')

# Codificação das variáveis de Cor de Prova para números inteiros
encoder = LabelEncoder()
for c in ['CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT']:
    df[c] = encoder.fit_transform(df[c])
    df_test[c] = encoder.transform(df_test[c])

# As observações que não possuem variáveis resposta (NU_NOTA_MT) serão desconsideradas
df = df[~df['NU_NOTA_MT'].isnull()]
df = df[cols]

# Variáveis ignoradas
drops = ['TP_SEXO', 
         'TP_COR_RACA',
         'TP_ENSINO',
         'TP_DEPENDENCIA_ADM_ESC',
         'NU_INSCRICAO',
         'CO_UF_RESIDENCIA',
         'TP_PRESENCA_CN',
         'TP_PRESENCA_CH', 
         'TP_PRESENCA_LC',
         'IN_CEGUEIRA']

# Variáveis Discretas
discrete = ['NU_NOTA_CN', 
            'NU_NOTA_CH', 
            'NU_NOTA_LC',
            'NU_NOTA_COMP1', 
            'NU_NOTA_COMP2', 
            'NU_NOTA_COMP3',
            'NU_NOTA_COMP4', 
            'NU_NOTA_COMP5', 
            'NU_NOTA_REDACAO']

# Variáveis categóricas
categorical = ['SG_UF_RESIDENCIA',
               'NU_IDADE',
               'TP_NACIONALIDADE',
               'TP_ST_CONCLUSAO',
               'TP_ANO_CONCLUIU',
               'TP_ESCOLA',
               'CO_PROVA_CN', 
               'CO_PROVA_CH',
               'CO_PROVA_LC', 
               'CO_PROVA_MT',
               'TP_LINGUA',
               'TP_STATUS_REDACAO',
               'Q001', 
               'Q002',
               'Q006',
               'Q024',
               'Q026', 
               'Q027', 
               'Q047']

# Variáveis binárias
binaries = ['IN_TREINEIRO', 
            'IN_BAIXA_VISAO',
            'IN_CEGUEIRA', 
            'IN_SURDEZ', 
            'IN_DISLEXIA', 
            'IN_DISCALCULIA',
            'IN_SABATISTA', 
            'IN_GESTANTE', 
            'IN_IDOSO',
            'Q025']

# Tratamento das variáveis nas bases de treino e teste
df.drop(columns=drops,inplace=True)
drops.remove('NU_INSCRICAO')
df_test.drop(columns=drops,inplace=True)

df['Q025'] = df['Q025'].apply(lambda x: 0 if x == 'A' else 1)
df_test['Q025'] = df_test['Q025'].apply(lambda x: 0 if x == 'A' else 1)

df['Q027'] = df['Q027'].fillna('other')
df_test['Q027'] = df_test['Q027'].fillna('other')

df[['NU_NOTA_CN','NU_NOTA_CH']] = df[['NU_NOTA_CN','NU_NOTA_CH']].fillna(0)
df_test = df_test.fillna(0)

df['NU_IDADE'] = pd.cut(df['NU_IDADE'],bins=[0,18,25,35,45,np.inf], labels=['A','B','C','D','E'])
df_test['NU_IDADE'] = pd.cut(df_test['NU_IDADE'],bins=[0,18,25,35,45,np.inf], labels=['A','B','C','D','E'])

df[categorical] = df[categorical].astype(str)  
df_test[categorical] = df_test[categorical].astype(str)

df = pd.get_dummies(df,columns=categorical)
df_test = pd.get_dummies(df_test,columns=categorical)

In [41]:
# Análise das variáveis que influência a variância da variável resposta
x = df.drop(columns='NU_NOTA_MT')
y = df['NU_NOTA_MT']

# Dataframe com R-quadrado das variáveis
rsquared_df = df.corr()['NU_NOTA_MT']
rsquared_df = (rsquared_df * rsquared_df).round(2)
rsquared_df = rsquared_df.to_frame()

# Dataframe com o p-valor do teste F 
score = f_regression(x,y)
score_df = pd.DataFrame({'pvalue':list((map(lambda x: round(x,10),score[1])))},index=x.columns)

# R quadrado e p-valor de cada variável
features = rsquared_df.merge(score_df,how='inner',left_index=True,right_index=True).sort_values(by='NU_NOTA_MT',ascending=False)
features['NU_NOTA_MT'].value_counts()

0.0000000000    130
0.0100000000     23
0.0200000000      6
0.1200000000      4
0.0400000000      4
0.0300000000      3
0.2400000000      2
0.0500000000      2
0.1100000000      2
0.3000000000      1
0.0600000000      1
0.1400000000      1
0.0700000000      1
0.0900000000      1
Name: NU_NOTA_MT, dtype: int64

In [45]:
# Cross Validation de diversos modelos de Regressão Linear.
for i in sorted(list(features['NU_NOTA_MT'].value_counts().index))[:-1]:
    list_features = list(features[features['NU_NOTA_MT'] > i].index)

    model = LinearRegression()

    cv_results = cross_validate(model,x[list_features],y,scoring=['neg_mean_squared_error','neg_mean_squared_log_error','r2'])
    print(f'R Quadrado das variáveis > {i}')
    print(f"neg_mean_squared_error: {cv_results['test_neg_mean_squared_error'].mean()}")
    print(f"neg_mean_squared_log_error: {cv_results['test_neg_mean_squared_log_error'].mean()}")
    print(f"r2: {cv_results['test_r2'].mean()}\n")
    

R Quadrado das variáveis > 0.0
neg_mean_squared_error: -5930.1768994586155
neg_mean_squared_log_error: -0.055354960035597214
r2: 0.4045550594313646

R Quadrado das variáveis > 0.01
neg_mean_squared_error: -5965.347324074989
neg_mean_squared_log_error: -0.05534246962425515
r2: 0.40095281885423883

R Quadrado das variáveis > 0.02
neg_mean_squared_error: -5990.304062858963
neg_mean_squared_log_error: -0.055329964130789236
r2: 0.3984638535132731

R Quadrado das variáveis > 0.03
neg_mean_squared_error: -6018.633722374836
neg_mean_squared_log_error: -0.055350010688089105
r2: 0.39563890778709776

R Quadrado das variáveis > 0.04
neg_mean_squared_error: -6058.454176201734
neg_mean_squared_log_error: -0.055330320893195374
r2: 0.3916075516855435

R Quadrado das variáveis > 0.05
neg_mean_squared_error: -6071.002548438785
neg_mean_squared_log_error: -0.05520310277402919
r2: 0.39033279166888357

R Quadrado das variáveis > 0.06
neg_mean_squared_error: -6078.663748343119
neg_mean_squared_log_error: -0

In [21]:
# Melhor modelo de Regressão Linear identificado
final_model = LinearRegression()

list_features = list(features[features['NU_NOTA_MT'] > 0.0].index)
final_model.fit(x[list_features],y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [99]:
# Execução do Modelo na base de teste
df_test['NU_NOTA_MT'] = final_model.predict(df_test[list_features])
df_test['NU_NOTA_MT'] = df_test['NU_NOTA_MT'].apply(lambda x: round(x,1))

#df_test[['NU_INSCRICAO','NU_NOTA_MT']].to_csv(r'answer.csv',index=False)

In [30]:
# Cross validation de XGBoost com Grid Search de parâmetros aleatórios
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=42)

params = {
    "colsample_bytree": [uniform(0.7, 0.3) for i in range(40)],
    "gamma": [uniform(0, 0.5) for i in range(40)],
    "learning_rate": [uniform(0.03, 0.3) for i in range(40)], 
    "max_depth": list(range(2,7)),
    "n_estimators": [randint(100, 150) for i in range(40)],
    "subsample": [uniform(0.6, 0.4) for i in range(40)]
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, scoring='neg_mean_squared_error', random_state=42, n_iter=120, cv=5, verbose=1, n_jobs=1, return_train_score=True)

search.fit(x[list_features], y)


Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 16.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=42, reg...
                                                      0.4603847214648239,
                                                      0.5962701273150146,
                                        

In [39]:
# Execução do XGBoost com os melhores parâmetros na base de teste e geração do arquivo de respostas do desafio
df_test['NU_NOTA_MT'] = search.best_estimator_.predict(df_test[list_features])
df_test['NU_NOTA_MT'] = df_test['NU_NOTA_MT'].apply(lambda x: round(x,1))

df_test[['NU_INSCRICAO','NU_NOTA_MT']].to_csv(r'answer.csv',index=False)